# Enhanced SageMaker Ground Truth Labeling Job Creation

This notebook provides a comprehensive interface for creating, managing, and monitoring SageMaker Ground Truth labeling jobs for drone imagery object detection.

## Overview

SageMaker Ground Truth helps you build high-quality training datasets for your machine learning models. This enhanced notebook will guide you through:

1. Setting up your data in S3
2. Configuring a labeling job with advanced options
3. Selecting and managing your workforce
4. Monitoring job progress with detailed metrics
5. Validating annotation quality
6. Converting Ground Truth output to YOLOv11 format
7. Visualizing labeled data
8. Managing labeling costs

## Prerequisites

- AWS account with appropriate permissions
- Access to SageMaker Studio
- Drone imagery dataset in S3 bucket
- `ground_truth_utils.py` module installed

## 1. Import Required Libraries

In [ ]:
import boto3
import sagemaker
import pandas as pd
import numpy as np
import json
import os
import datetime
import time
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output, Image as IPImage
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import io
from PIL import Image, ImageDraw, ImageFont
import re
import uuid
import warnings
warnings.filterwarnings('ignore')

# Import our custom Ground Truth utilities
from src.data.ground_truth_utils import (
    create_labeling_job_config,
    monitor_labeling_job,
    get_labeling_job_metrics,
    convert_ground_truth_to_yolo,
    validate_annotation_quality,
    estimate_labeling_cost,
    create_labeling_instructions,
    create_manifest_file,
    list_labeling_jobs,
    visualize_annotations
)

# Set up AWS session with 'ab' profile
session = boto3.Session(profile_name='ab')
sagemaker_client = session.client('sagemaker')
s3_client = session.client('s3')

# Initialize SageMaker session
sagemaker_session = sagemaker.Session(boto_session=session)

# Set default bucket
bucket = "lucaskle-ab3-project-pv"
prefix = "ground-truth-jobs"

# Get SageMaker execution role
role = sagemaker.get_execution_role()

## 2. Create Dashboard Interface

Let's create a dashboard interface to navigate through the different steps of the labeling job creation process.

In [ ]:
# Create tab widget for the dashboard
dashboard = widgets.Tab()

# Create tabs for each step
data_setup_tab = widgets.VBox()
job_config_tab = widgets.VBox()
cost_estimation_tab = widgets.VBox()
job_monitoring_tab = widgets.VBox()
annotation_validation_tab = widgets.VBox()
conversion_tab = widgets.VBox()
visualization_tab = widgets.VBox()

# Set tab titles
dashboard.children = [data_setup_tab, job_config_tab, cost_estimation_tab, job_monitoring_tab, 
                      annotation_validation_tab, conversion_tab, visualization_tab]
dashboard.set_title(0, 'Data Setup')
dashboard.set_title(1, 'Job Configuration')
dashboard.set_title(2, 'Cost Estimation')
dashboard.set_title(3, 'Job Monitoring')
dashboard.set_title(4, 'Validation')
dashboard.set_title(5, 'YOLO Conversion')
dashboard.set_title(6, 'Visualization')

# Display the dashboard
display(dashboard)

## 3. Data Setup

First, we need to prepare our dataset for labeling. Ground Truth requires a manifest file that lists all the images to be labeled.

In [ ]:
# Create interactive widgets for data setup tab
bucket_input = widgets.Text(
    value=bucket,
    description='S3 Bucket:',
    style={'description_width': 'initial'}
)

prefix_input = widgets.Text(
    value="raw-images/",
    description='Image Prefix:',
    style={'description_width': 'initial'}
)

max_images = widgets.IntSlider(
    value=100,
    min=10,
    max=1000,
    step=10,
    description='Max Images:',
    style={'description_width': 'initial'}
)

image_filter = widgets.Text(
    value="",
    placeholder="Filter by filename (e.g., 'drone')",
    description='Filter:',
    style={'description_width': 'initial'}
)

# Create button to list images
list_button = widgets.Button(
    description='List Images',
    button_style='info',
    icon='search'
)

# Create output area for image listing
list_output = widgets.Output()

# Create button to create manifest
manifest_button = widgets.Button(
    description='Create Manifest',
    button_style='success',
    icon='file-upload',
    disabled=True
)

# Create output area for manifest creation
manifest_output = widgets.Output()

# Create button to preview images
preview_button = widgets.Button(
    description='Preview Images',
    button_style='warning',
    icon='image',
    disabled=True
)

# Create output area for image preview
preview_output = widgets.Output()

# Function to list images in S3
def list_images(b=None):
    with list_output:
        clear_output()
        
        # Define the S3 path where your images are stored
        image_path = f"s3://{bucket_input.value}/{prefix_input.value}"
        
        print(f"Listing images in {image_path}...")
        
        # List all images in the bucket
        response = s3_client.list_objects_v2(
            Bucket=bucket_input.value,
            Prefix=prefix_input.value
        )
        
        # Filter for image files
        image_extensions = [".jpg", ".jpeg", ".png"]
        image_files = []
        
        if 'Contents' in response:
            for obj in response['Contents']:
                key = obj['Key']
                if any(key.lower().endswith(ext) for ext in image_extensions):
                    # Apply filter if provided
                    if image_filter.value and image_filter.value.lower() not in key.lower():
                        continue
                    image_files.append(f"s3://{bucket_input.value}/{key}")
        
        # Limit to max_images
        image_files = image_files[:max_images.value]
        
        print(f"Found {len(image_files)} images for labeling")
        
        # Display the first few images
        if image_files:
            print("\nFirst 5 images:")
            for i, image in enumerate(image_files[:5]):
                print(f"  {i+1}. {os.path.basename(image)}")
            
            # Enable the manifest and preview buttons
            manifest_button.disabled = False
            preview_button.disabled = False
            
            # Store the image files in a global variable
            global selected_image_files
            selected_image_files = image_files
        else:
            print("No images found matching the criteria.")
            manifest_button.disabled = True
            preview_button.disabled = True

# Function to create manifest file
def create_manifest(b=None):
    with manifest_output:
        clear_output()
        
        # Generate a unique job name
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
        job_name = f"drone-detection-{timestamp}"
        
        print(f"Creating manifest file for job: {job_name}")
        print(f"Processing {len(selected_image_files)} images...")
        
        # Create the manifest file
        manifest_uri = create_manifest_file(
            image_files=selected_image_files,
            output_bucket=bucket_input.value,
            output_prefix=prefix,
            job_name=job_name
        )
        
        print(f"Manifest file created at: {manifest_uri}")
        
        # Store the manifest URI and job name in global variables
        global manifest_uri_global, job_name_global
        manifest_uri_global = manifest_uri
        job_name_global = job_name
        
        # Update the job name input in the job configuration tab
        job_name_input.value = job_name
        
        # Switch to the job configuration tab
        dashboard.selected_index = 1

# Function to preview images
def preview_images(b=None):
    with preview_output:
        clear_output()
        
        # Select a few images to preview
        preview_count = min(3, len(selected_image_files))
        preview_images = selected_image_files[:preview_count]
        
        print(f"Previewing {preview_count} images...")
        
        # Create a figure with subplots
        fig, axes = plt.subplots(1, preview_count, figsize=(15, 5))
        if preview_count == 1:
            axes = [axes]  # Make it iterable for a single image
        
        # Download and display each image
        for i, image_uri in enumerate(preview_images):
            try:
                # Parse the S3 URI
                bucket_name = image_uri.split('/')[2]
                key = '/'.join(image_uri.split('/')[3:])
                
                # Download the image to a temporary file
                temp_file = f"/tmp/preview_{i}.jpg"
                s3_client.download_file(bucket_name, key, temp_file)
                
                # Display the image
                img = plt.imread(temp_file)
                axes[i].imshow(img)
                axes[i].set_title(os.path.basename(key))
                axes[i].axis('off')
                
                # Clean up
                os.remove(temp_file)
            except Exception as e:
                print(f"Error displaying image {image_uri}: {str(e)}")
        
        plt.tight_layout()
        plt.show()

# Connect button click events
list_button.on_click(list_images)
manifest_button.on_click(create_manifest)
preview_button.on_click(preview_images)

# Assemble the data setup tab
data_setup_tab.children = [
    widgets.HTML("<h3>Step 1: Configure Data Source</h3>"),
    widgets.HBox([bucket_input, prefix_input]),
    widgets.HBox([max_images, image_filter]),
    widgets.HBox([list_button, preview_button, manifest_button]),
    widgets.HTML("<h3>Image Listing</h3>"),
    list_output,
    widgets.HTML("<h3>Image Preview</h3>"),
    preview_output,
    widgets.HTML("<h3>Manifest Creation</h3>"),
    manifest_output
]

## 4. Job Configuration

Now we'll configure our labeling job with advanced options.

In [ ]:
# Create interactive widgets for job configuration tab
job_name_input = widgets.Text(
    value=f"drone-detection-{datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S')}",
    description='Job Name:',
    style={'description_width': 'initial'}
)

task_type = widgets.Dropdown(
    options=['BoundingBox', 'ImageClassification'],
    value='BoundingBox',
    description='Task Type:',
    style={'description_width': 'initial'}
)

worker_type = widgets.Dropdown(
    options=['private', 'public'],
    value='private',
    description='Worker Type:',
    style={'description_width': 'initial'}
)

# For the categories, we'll use a simple text input with comma-separated values
categories_input = widgets.Text(
    value='drone, vehicle, person, building',
    description='Categories:',
    style={'description_width': 'initial'}
)

instructions_input = widgets.Textarea(
    value='Label all drones and other objects visible in the image.',
    description='Instructions:',
    layout={'width': '100%', 'height': '100px'},
    style={'description_width': 'initial'}
)

detailed_instructions = widgets.Checkbox(
    value=True,
    description='Include detailed instructions',
    style={'description_width': 'initial'}
)

max_budget = widgets.FloatSlider(
    value=100.0,
    min=10.0,
    max=1000.0,
    step=10.0,
    description='Max Budget ($):',
    style={'description_width': 'initial'}
)

# Create button to generate instructions preview
preview_instructions_button = widgets.Button(
    description='Preview Instructions',
    button_style='info',
    icon='eye'
)

# Create output area for instructions preview
instructions_preview_output = widgets.Output()

# Create button to configure job
job_config_button = widgets.Button(
    description='Configure Job',
    button_style='success',
    icon='cog'
)

# Create output area for job configuration
job_config_output = widgets.Output()

# Function to preview instructions
def preview_instructions(b=None):
    with instructions_preview_output:
        clear_output()
        
        # Parse categories from comma-separated string
        categories = [cat.strip() for cat in categories_input.value.split(',') if cat.strip()]
        
        # Generate instructions
        instructions_html = create_labeling_instructions(
            task_type=task_type.value,
            categories=categories,
            detailed_instructions=detailed_instructions.value,
            custom_instructions=instructions_input.value
        )
        
        # Display instructions
        display(HTML(instructions_html))

# Function to configure job
def configure_job(b=None):
    with job_config_output:
        clear_output()
        
        # Check if manifest URI is available
        if 'manifest_uri_global' not in globals():
            print("Error: No manifest file available. Please create a manifest file first.")
            return
        
        # Parse categories from comma-separated string
        categories = [cat.strip() for cat in categories_input.value.split(',') if cat.strip()]
        
        # Define output path
        output_path = f"s3://{bucket_input.value}/{prefix}/output/"
        
        # Generate instructions
        instructions_html = create_labeling_instructions(
            task_type=task_type.value,
            categories=categories,
            detailed_instructions=detailed_instructions.value,
            custom_instructions=instructions_input.value
        )
        
        print(f"Configuring labeling job: {job_name_input.value}")
        print(f"Task type: {task_type.value}")
        print(f"Worker type: {worker_type.value}")
        print(f"Categories: {categories}")
        print(f"Max budget: ${max_budget.value:.2f}")
        
        # Create labeling job configuration
        try:
            labeling_job_config = create_labeling_job_config(
                job_name=job_name_input.value,
                input_path=manifest_uri_global,
                output_path=output_path,
                task_type=task_type.value,
                worker_type=worker_type.value,
                labels=categories,
                instructions=instructions_html,
                max_budget_usd=max_budget.value,
                role_arn=role
            )
            
            print("\nJob configuration created successfully!")
            
            # Store the job configuration in a global variable
            global labeling_job_config_global
            labeling_job_config_global = labeling_job_config
            
            # Update the cost estimation tab
            update_cost_estimate()
            
            # Switch to the cost estimation tab
            dashboard.selected_index = 2
            
        except Exception as e:
            print(f"Error creating job configuration: {str(e)}")

# Connect button click events
preview_instructions_button.on_click(preview_instructions)
job_config_button.on_click(configure_job)

# Assemble the job configuration tab
job_config_tab.children = [
    widgets.HTML("<h3>Step 2: Configure Labeling Job</h3>"),
    widgets.HBox([job_name_input, task_type]),
    widgets.HBox([worker_type, max_budget]),
    categories_input,
    widgets.HTML("<h4>Worker Instructions</h4>"),
    instructions_input,
    widgets.HBox([detailed_instructions, preview_instructions_button]),
    instructions_preview_output,
    widgets.HTML("<h4>Job Configuration</h4>"),
    job_config_button,
    job_config_output
]

## 5. Cost Estimation

Let's estimate the cost of our labeling job to ensure it stays within budget.

In [ ]:
# Create interactive widgets for cost estimation tab
objects_per_image = widgets.FloatSlider(
    value=3.0,
    min=1.0,
    max=10.0,
    step=0.5,
    description='Objects per Image:',
    style={'description_width': 'initial'}
)

# Create button to update cost estimate
update_cost_button = widgets.Button(
    description='Update Estimate',
    button_style='info',
    icon='refresh'
)

# Create output area for cost estimation
cost_output = widgets.Output()

# Create button to start labeling job
start_job_button = widgets.Button(
    description='Start Labeling Job',
    button_style='success',
    icon='play'
)

# Create output area for job start
start_job_output = widgets.Output()

# Function to update cost estimate
def update_cost_estimate(b=None):
    with cost_output:
        clear_output()
        
        # Check if we have image files
        if 'selected_image_files' not in globals():
            print("Error: No images selected. Please select images first.")
            return
        
        # Get the number of images
        num_images = len(selected_image_files)
        
        # Estimate cost
        cost_estimate = estimate_labeling_cost(
            num_images=num_images,
            task_type=task_type.value,
            worker_type=worker_type.value,
            objects_per_image=objects_per_image.value
        )
        
        # Display cost estimate
        print(f"Cost Estimate for {num_images} images:")
        print(f"Base Cost: ${cost_estimate['base_cost']:.2f}")
        print(f"Adjusted Cost: ${cost_estimate['adjusted_cost']:.2f}")
        print(f"Service Cost: ${cost_estimate['service_cost']:.2f}")
        print(f"Storage Cost: ${cost_estimate['storage_cost']:.2f}")
        print(f"Data Transfer Cost: ${cost_estimate['data_transfer_cost']:.2f}")
        print(f"Total Estimated Cost: ${cost_estimate['total_cost']:.2f}")
        print(f"Cost per Image: ${cost_estimate['cost_per_image']:.4f}")
        
        # Display cost breakdown
        print("\nCost Breakdown:")
        labels = ['Labeling', 'Service', 'Storage', 'Data Transfer']
        sizes = [
            cost_estimate['adjusted_cost'],
            cost_estimate['service_cost'],
            cost_estimate['storage_cost'],
            cost_estimate['data_transfer_cost']
        ]
        
        # Create a pie chart
        fig, ax = plt.subplots(figsize=(8, 6))
        ax.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90)
        ax.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle
        plt.title('Cost Breakdown')
        plt.show()
        
        # Display cost control recommendations
        if 'cost_control_recommendations' in cost_estimate and cost_estimate['cost_control_recommendations']:
            print("\nCost Control Recommendations:")
            for i, recommendation in enumerate(cost_estimate['cost_control_recommendations']):
                print(f"{i+1}. {recommendation}")
        
        # Enable the start job button if the cost is within budget
        if cost_estimate['total_cost'] <= max_budget.value:
            start_job_button.disabled = False
            print(f"\n✅ Estimated cost (${cost_estimate['total_cost']:.2f}) is within budget (${max_budget.value:.2f})")
        else:
            start_job_button.disabled = True
            print(f"\n❌ Warning: Estimated cost (${cost_estimate['total_cost']:.2f}) exceeds budget (${max_budget.value:.2f})")

# Function to start labeling job
def start_labeling_job(b=None):
    with start_job_output:
        clear_output()
        
        # Check if job configuration is available
        if 'labeling_job_config_global' not in globals():
            print("Error: No job configuration available. Please configure the job first.")
            return
        
        print(f"Starting labeling job: {job_name_input.value}")
        
        try:
            # Create the labeling job
            response = sagemaker_client.create_labeling_job(**labeling_job_config_global)
            
            # Get the labeling job ARN
            labeling_job_arn = response['LabelingJobArn']
            
            print(f"Labeling job created successfully!")
            print(f"Job ARN: {labeling_job_arn}")
            
            # Store the job name in a global variable
            global active_job_name
            active_job_name = job_name_input.value
            
            # Update the job monitoring tab
            job_name_monitor.value = active_job_name
            
            # Switch to the job monitoring tab
            dashboard.selected_index = 3
            
        except Exception as e:
            print(f"Error starting labeling job: {str(e)}")

# Connect button click events
update_cost_button.on_click(update_cost_estimate)
start_job_button.on_click(start_labeling_job)

# Assemble the cost estimation tab
cost_estimation_tab.children = [
    widgets.HTML("<h3>Step 3: Estimate Labeling Costs</h3>"),
    widgets.HBox([objects_per_image, update_cost_button]),
    cost_output,
    widgets.HTML("<h4>Start Labeling Job</h4>"),
    start_job_button,
    start_job_output
]

## 6. Job Monitoring

Now we can monitor the progress of our labeling job with detailed metrics.

In [ ]:
# Create interactive widgets for job monitoring tab
job_name_monitor = widgets.Text(
    value="",
    description='Job Name:',
    style={'description_width': 'initial'}
)

# Create button to monitor job
monitor_job_button = widgets.Button(
    description='Monitor Job',
    button_style='info',
    icon='search'
)

# Create output area for job monitoring
monitor_job_output = widgets.Output()

# Create button to get detailed metrics
detailed_metrics_button = widgets.Button(
    description='Detailed Metrics',
    button_style='warning',
    icon='chart-bar',
    disabled=True
)

# Create output area for detailed metrics
detailed_metrics_output = widgets.Output()

# Create auto-refresh checkbox
auto_refresh = widgets.Checkbox(
    value=False,
    description='Auto-refresh (30s)',
    style={'description_width': 'initial'}
)

# Create output area for refresh timer
refresh_timer_output = widgets.Output()

# Create button to list all jobs
list_jobs_button = widgets.Button(
    description='List All Jobs',
    button_style='primary',
    icon='list'
)

# Create output area for job listing
list_jobs_output = widgets.Output()

# Function to monitor job
def monitor_job(b=None):
    with monitor_job_output:
        clear_output()
        
        # Check if job name is provided
        if not job_name_monitor.value:
            print("Error: No job name provided. Please enter a job name.")
            return
        
        print(f"Monitoring labeling job: {job_name_monitor.value}")
        
        try:
            # Get job status
            status = monitor_labeling_job(job_name_monitor.value, sagemaker_client)
            
            # Display job status
            print(f"\nJob Status: {status['LabelingJobStatus']}")
            print(f"Creation Time: {status['CreationTime']}")
            print(f"Last Modified: {status['LastModifiedTime']}")
            
            # Display label counters
            print(f"\nLabel Counters:")
            print(f"Total Objects: {status['LabelCounters']['TotalObjects']}")
            print(f"Labeled Objects: {status['LabelCounters']['LabeledObjects']}")
            print(f"Failed Objects: {status['LabelCounters']['FailedNonRetryableObjects']}")
            print(f"Unlabeled Objects: {status['LabelCounters']['UnlabeledObjects']}")
            
            # Display completion percentage
            if 'CompletionPercentage' in status:
                print(f"\nCompletion: {status['CompletionPercentage']:.2f}%")
                
                # Create a progress bar
                plt.figure(figsize=(10, 1))
                plt.barh(0, status['CompletionPercentage'], color='green')
                plt.barh(0, 100 - status['CompletionPercentage'], left=status['CompletionPercentage'], color='lightgray')
                plt.xlim(0, 100)
                plt.yticks([])
                plt.title(f"Completion: {status['CompletionPercentage']:.2f}%")
                plt.show()
            
            # Display labeling speed
            if 'LabelingSpeed' in status:
                print(f"Labeling Speed: {status['LabelingSpeed']:.2f} objects/hour")
            
            # Display estimated completion time
            if 'EstimatedCompletionTime' in status and status['EstimatedCompletionTime']:
                print(f"Estimated Completion: {status['EstimatedCompletionTime']}")
            
            # Display cost information
            if 'EstimatedTotalCost' in status:
                print(f"\nEstimated Cost: ${status['EstimatedTotalCost']:.2f}")
                print(f"Max Budget: ${status.get('MaxBudgetUSD', 'N/A')}")
            
            # Enable detailed metrics button if job is in progress or completed
            if status['LabelingJobStatus'] in ['InProgress', 'Completed']:
                detailed_metrics_button.disabled = False
            else:
                detailed_metrics_button.disabled = True
            
            # If job is completed, display completion message
            if status['LabelingJobStatus'] == 'Completed':
                print("\n✅ Job completed successfully!")
                print("You can now proceed to the Validation tab to validate the annotations.")
                
                # Disable auto-refresh
                auto_refresh.value = False
            
            # If job failed, display error message
            elif status['LabelingJobStatus'] == 'Failed':
                print(f"\n❌ Job failed: {status.get('FailureReason', 'Unknown error')}")
                
                # Disable auto-refresh
                auto_refresh.value = False
            
            # If job is in progress, display auto-refresh message
            elif status['LabelingJobStatus'] == 'InProgress':
                if auto_refresh.value:
                    print("\nAuto-refreshing in 30 seconds...")
        
        except Exception as e:
            print(f"Error monitoring job: {str(e)}")

# Function to get detailed metrics
def get_detailed_metrics(b=None):
    with detailed_metrics_output:
        clear_output()
        
        # Check if job name is provided
        if not job_name_monitor.value:
            print("Error: No job name provided. Please enter a job name.")
            return
        
        print(f"Getting detailed metrics for job: {job_name_monitor.value}")
        
        try:
            # Get detailed metrics
            metrics = get_labeling_job_metrics(job_name_monitor.value, sagemaker_client)
            
            # Display annotation metrics
            if 'annotation_metrics' in metrics and metrics['annotation_metrics']:
                print("\nAnnotation Metrics:")
                
                # Display label distribution
                if 'label_distribution' in metrics['annotation_metrics']:
                    print("\nLabel Distribution:")
                    label_dist = metrics['annotation_metrics']['label_distribution']
                    
                    # Create a bar chart
                    plt.figure(figsize=(10, 5))
                    plt.bar(label_dist.keys(), label_dist.values())
                    plt.title('Label Distribution')
                    plt.xlabel('Label')
                    plt.ylabel('Count')
                    plt.xticks(rotation=45)
                    plt.tight_layout()
                    plt.show()
                
                # Display annotations per image
                if 'avg_annotations_per_image' in metrics['annotation_metrics']:
                    print(f"Average Annotations per Image: {metrics['annotation_metrics']['avg_annotations_per_image']:.2f}")
                    print(f"Min Annotations per Image: {metrics['annotation_metrics'].get('min_annotations_per_image', 'N/A')}")
                    print(f"Max Annotations per Image: {metrics['annotation_metrics'].get('max_annotations_per_image', 'N/A')}")
            
            # Display time metrics
            if 'time_metrics' in metrics and metrics['time_metrics']:
                print("\nTime Metrics:")
                print(f"Total Time: {metrics['time_metrics']['total_time_hours']:.2f} hours")
                print(f"Average Time per Object: {metrics['time_metrics']['average_time_per_object']:.2f} seconds")
            
            # Display cost metrics
            if 'cost_metrics' in metrics and metrics['cost_metrics']:
                print("\nCost Metrics:")
                print(f"Max Budget: ${metrics['cost_metrics']['max_budget_usd']:.2f}")
                print(f"Estimated Total Cost: ${metrics['cost_metrics']['estimated_total_cost']:.2f}")
                print(f"Cost per Object: ${metrics['cost_metrics']['cost_per_object']:.4f}")
        
        except Exception as e:
            print(f"Error getting detailed metrics: {str(e)}")

# Function to list all labeling jobs
def list_all_jobs(b=None):
    with list_jobs_output:
        clear_output()
        
        print("Listing all labeling jobs...")
        
        try:
            # List all labeling jobs
            response = sagemaker_client.list_labeling_jobs()
            
            # Display job list
            if 'LabelingJobSummaryList' in response and response['LabelingJobSummaryList']:
                jobs = response['LabelingJobSummaryList']
                
                print(f"Found {len(jobs)} labeling jobs:\n")
                
                # Create a table
                data = []
                for job in jobs:
                    data.append({
                        'Job Name': job['LabelingJobName'],
                        'Status': job['LabelingJobStatus'],
                        'Creation Time': job['CreationTime'].strftime('%Y-%m-%d %H:%M:%S'),
                        'Labeled/Total': f"{job['LabelCounters']['LabeledObjects']}/{job['LabelCounters']['TotalObjects']}"
                    })
                
                # Display as DataFrame
                df = pd.DataFrame(data)
                display(df)
                
                # Create a dropdown to select a job
                job_selector = widgets.Dropdown(
                    options=[job['LabelingJobName'] for job in jobs],
                    description='Select Job:',
                    style={'description_width': 'initial'}
                )
                
                # Function to select job
                def select_job(change):
                    job_name_monitor.value = change['new']
                    monitor_job(None)
                
                job_selector.observe(select_job, names='value')
                
                display(job_selector)
            else:
                print("No labeling jobs found.")
        
        except Exception as e:
            print(f"Error listing jobs: {str(e)}")

# Connect button click events
monitor_job_button.on_click(monitor_job)
detailed_metrics_button.on_click(get_detailed_metrics)
list_jobs_button.on_click(list_all_jobs)

# Assemble the job monitoring tab
job_monitoring_tab.children = [
    widgets.HTML("<h3>Step 4: Monitor Labeling Job</h3>"),
    widgets.HBox([job_name_monitor, monitor_job_button, auto_refresh]),
    monitor_job_output,
    widgets.HTML("<h4>Detailed Metrics</h4>"),
    detailed_metrics_button,
    detailed_metrics_output,
    widgets.HTML("<h4>All Labeling Jobs</h4>"),
    list_jobs_button,
    list_jobs_output
]